<a href="https://colab.research.google.com/github/rseliman/JoaoMagic/blob/main/Letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list contents in yolov4-tiny folder in your drive
!ls /mydrive/Joao

In [ ]:
%cd /content/
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# build darknet
!make

In [ ]:
# Clean the data and cfg folders first except the labels folder in data which is required

%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

In [ ]:
#copy the datasets zip file to the root darknet folder
!cp -r /content/gdrive/MyDrive/Joao /content/darknet/data/



In [ ]:
#copy the custom cfg file from the drive to the darknet/cfg folder
!cp /content/gdrive/MyDrive/Joao/yolov4-tiny-custom.cfg ./cfg

In [ ]:
# copy the obj.names and obj.data files so that they are now in /darknet/data/ folder
!cp /content/gdrive/MyDrive/Joao/obj.names ./data
!cp /content/gdrive/MyDrive/Joao/obj.data  ./data

In [ ]:
#copy the process.py file from the drive to the darknet directory
!cp /mydrive/Joao/process.py ./

In [ ]:
%ls

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip /content/darknet/data/JoaoMagic/PlayingCardsv2.zip -d /content/darknet/data/JoaoMagic/

In [ ]:
# run process.py ( this creates the train.txt and test.txt files in our darknet/data folder )
!python process.py

# list the contents of data folder to check if the train.txt and test.txt files have been created
!ls data/

In [ ]:
import os

# Directorio que contiene las imágenes de entrenamiento y prueba
base_dir = '/content/darknet/data/Joao/'

# Directorios específicos para entrenamiento y prueba
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Obtener las rutas de las imágenes de entrenamiento y prueba
train_image_paths = [os.path.join(train_dir, img) for img in os.listdir(train_dir) if img.endswith('.jpg')]
test_image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir) if img.endswith('.jpg')]

# Crear y guardar el archivo train.txt
with open('/content/darknet/data/train.txt', 'w') as f:
    for path in train_image_paths:
        f.write(path + '\n')

# Crear y guardar el archivo test.txt
with open('/content/darknet/data/test.txt', 'w') as f:
    for path in test_image_paths:
        f.write(path + '\n')

In [ ]:

# Download the yolov4-tiny pre-trained weights file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture

!./darknet detector train data/obj.data cfg/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show -weights /gdrive/MyDrive/Joao/training/yolov4-tiny-custom_last.weights


In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False

In [ ]:
#to restart training your custom detector where you left off(using the weights that were saved last)

!./darknet detector train data/obj.data cfg/yolov4-tiny-custom.cfg /content/gdrive/MyDrive/Joao/training/yolov4-tiny-custom_10000.weights -dont_show -map

In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  #plt.show('')


In [ ]:
#only works if the training does not get interrupted
imShow('chart.png')

In [ ]:
#You can check the mAP for all the saved weights to see which gives the best results ( xxxx here is the saved weight number like 4000, 5000 or 6000 snd so on )

!./darknet detector map data/obj.data cfg/yolov4-tiny-custom.cfg /mydrive/Joao/training/yolov4-tiny-custom_20000.weights -points 0